<a href="https://colab.research.google.com/github/Iispar/hlt-project/blob/main/course_project_2023_template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to HLT 2023 Project (Template)

- Student(s) Name(s): Iiro Partanen
- Date: -
- Chosen Corpus: emotion
- Contributions (if group project):

### Corpus information

- Description of the chosen corpus: 
Emotion is a dataset of English Twitter messages with six basic emotions: anger, fear, joy, love, sadness, and surprise. For more detailed information please refer to the paper.
- Paper(s) and other published materials related to the corpus: 
  - CARER: Contextualized Affect Representations for Emotion Recognition (Saravia et al., EMNLP 2018)
  - https://paperswithcode.com/dataset/emotion
- State-of-the-art performance (best published results) on this corpus: 95% f1

---

## 1. Setup

In [1]:
!pip3 install -q transformers datasets evaluate
!pip install trankit
!pip install optuna
import datasets
import sklearn.feature_extraction
import torch
import transformers
import numpy as np
import evaluate
import optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 59.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 26.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 67.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 18.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.or

---

## 2. Data download and preprocessing

### 2.1. Download the corpus

In [2]:
dset = datasets.load_dataset("emotion");
# check it works
print(dset);

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/16000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Dataset emotion downloaded and prepared to /root/.cache/huggingface/datasets/emotion/split/1.0.0/cca5efe2dfeb58c1d098e0f9eeb200e9927d889b5a03c67097275dfb5fe463bd. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
})


### 2.2. Preprocessing

In [3]:
# vectorizes one item
def vectorize_item(item):
    vectorized = vectorizer.transform([item["text"]]); # vectorize. Initialized below...
    non_zero_features = vectorized.nonzero()[1]; # get the nonzeros and we take only the columns of the nonzeros because our matrix is only one row.
    non_zero_features += 1; # index zero is for padding so let's avoid it by adding 1 to all.

    return {"input_ids":non_zero_features} 

In [4]:
dset.shuffle(); #shuffle dataset for safety

# vectorization.
vectorizer = sklearn.feature_extraction.text.CountVectorizer( # get the vectorizer.
    binary = True,
    max_features = 20000, # Selected 20k to start with.
    token_pattern = r"(?u)\b\w+\b" # Token pattern to include one char words.
    )

texts=[item["text"] for item in dset["train"]]; # get all texts from train
vectorizer.fit(texts); # fitting the vectorizer

# vectorize the whole dataset.
dset_tokenized = dset.map(vectorize_item,num_proc=4);
# check it works
print(dset_tokenized["train"][0]);

Map (num_proc=4):   0%|          | 0/16000 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/2000 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/2000 [00:00<?, ? examples/s]

{'text': 'i didnt feel humiliated', 'label': 0, 'input_ids': [3620, 4931, 6438, 6495]}


In [5]:
# padding and batching
def collator(list_of_items):
    allLabels = [item["label"] for item in list_of_items]; # list of all labels.
    batch = {"labels": torch.tensor(allLabels)}; # create a tenstor for the item (batch)
    tensors = [];
    max_len = max(len(item["input_ids"]) for item in list_of_items); # longest example in the batch. Pad to here.
    for item in list_of_items:
        ids = torch.tensor(item["input_ids"]); # input ids to tensor
        padded = torch.nn.functional.pad(ids,(0,max_len-ids.shape[0])); # actual padding. Pads ids, from + to max with 0.
        tensors.append(padded); # appended ids to tensors
    batch["input_ids"] = torch.vstack(tensors); # stacks items as they are now same len. Now these are matrixes.
    return batch;

# check it works
batch=collator([dset_tokenized["train"][2],dset_tokenized["train"][7]])
print("Shape of labels:",batch["labels"].shape)
print("Shape of input_ids:",batch["input_ids"].shape)
print(batch["labels"])
print(batch["input_ids"])
     

Shape of labels: torch.Size([2])
Shape of input_ids: torch.Size([2, 13])
tensor([3, 4])
tensor([[    1,  4931,  5739,  5800,  6495,  6563,  8456, 10157, 13643, 15061,
             0,     0,     0],
        [    1,    34,   749,  2680,  4931,  6495,  7076,  7712,  8076,  9245,
          9325, 13339, 15116]])


---

## 3. Machine learning model

### 3.1. Model training

In [6]:
# Your code to train the machine learning model on the training set and evaluate the performance on the validation set here

# needs a config, we wil just pass it.
class MLPConfig(transformers.PretrainedConfig):
    pass;

# model
class MLP(transformers.PreTrainedModel):
      config_class = MLPConfig; # sets config
      #initilazition
      def __init__(self,config):
        super().__init__(config); # call the super with out config which is now pass..
        self.vocab_size = config.vocab_size; # embedding matrix row count
        # Build embedding of vocab size +1 x hidden size. +1 again because of padding.
        self.embedding = torch.nn.Embedding(num_embeddings=self.vocab_size+1,embedding_dim=config.hidden_size,padding_idx=0);
        torch.nn.init.uniform_(self.embedding.weight.data,-0.001,0.001); # initialization of the embedding values
        self.output = torch.nn.Linear(in_features=config.hidden_size,out_features=config.nlabels); # output layer is the size of the labels x hidden size.

      # forward
      def forward(self,input_ids,labels=None):
        embedded = self.embedding(input_ids); # sum up all the embeddings
        embedded_summed = torch.sum(embedded,dim=1); # sum up across word dimension
        projected = torch.tanh(embedded_summed); # non-linearity
        logits = self.output(projected); # apply the outer layer
      

        ## calculates the loss
        if labels is not None:
            # calculates the loss.
            loss = torch.nn.CrossEntropyLoss();
            return (loss(logits,labels),logits);
        else:
            # if no labels, just return the logits
            return (logits,);
  
# config
mlp_config = MLPConfig(vocab_size=len(vectorizer.vocabulary_),hidden_size=25,nlabels=6);

In [7]:
# training

# Set training arguments
trainer_args = transformers.TrainingArguments(
    "mlp_checkpoints", #save checkpoints here
    evaluation_strategy = "steps",
    logging_strategy = "steps",
    eval_steps = 500,
    logging_steps = 500,
    learning_rate = 1e-4, #learning rate of the gradient descent
    max_steps = 20000,
    load_best_model_at_end = True,
    per_device_train_batch_size = 128
)


# evaluating
accuracy = evaluate.load("accuracy");
def compute_accuracy(outputs_and_labels):
    outputs, labels = outputs_and_labels;
    predictions = np.argmax(outputs, axis=-1); #pick the index of the "winning" label
    return accuracy.compute(predictions=predictions, references=labels);

# actual training
mlp = MLP(mlp_config); # Make a the actual model  
early_stopping = transformers.EarlyStoppingCallback(5); # stop training if the eval loss is not getting better.

# params
trainer = transformers.Trainer(
    model = mlp,
    args = trainer_args,
    train_dataset = dset_tokenized["train"],
    eval_dataset = dset_tokenized["test"],
    compute_metrics = compute_accuracy,
    data_collator = collator,
    callbacks = [early_stopping]
)

# FINALLY!
trainer.train();


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss,Accuracy
500,1.619600,1.516018,0.445500
1000,1.429600,1.374360,0.572500
1500,1.232800,1.208243,0.613000
2000,1.019900,1.043725,0.706000
2500,0.825000,0.897714,0.768000
3000,0.662100,0.775110,0.819500
3500,0.532400,0.675753,0.848500
4000,0.431700,0.597192,0.861000
4500,0.354300,0.535383,0.870000
5000,0.294700,0.487515,0.875500


### 3.2 Hyperparameter optimization

In [8]:
# Used optuna for optimization

def objective(trial):
    # Define the search space for hyperparameters
    learning_rate = trial.suggest_float("learning_rate", 1e-7, 1e-3, log=True)
    batch_size = trial.suggest_categorical("batch_size", [8, 16, 64, 128])
    epochs=trial.suggest_int('num_train_epochs', low = 2,high = 6),

    # params
    trainer_args = transformers.TrainingArguments(
        "mlp_checkpoints",
        evaluation_strategy = "steps",
        logging_strategy = "steps",
        eval_steps = 500,
        logging_steps = 500,
        learning_rate = learning_rate,
        max_steps = 30000,
        load_best_model_at_end = True,
        per_device_train_batch_size = batch_size,
        per_device_eval_batch_size = batch_size,
        num_train_epochs = epochs
    )

    # the model
    mlp = MLP(mlp_config)

    # train a model
    trainer = transformers.Trainer(
        model = mlp,
        args = trainer_args,
        train_dataset = dset_tokenized["train"],
        eval_dataset = dset_tokenized["validation"],
        compute_metrics = compute_accuracy,
        data_collator = collator,
        callbacks = [early_stopping]
    )

    trainer.train()
    eval_results = trainer.evaluate()
    return eval_results["eval_accuracy"] # return the best result.

study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=20)


[I 2023-04-29 10:20:02,644] A new study created in memory with name: no-name-14a89845-85ce-4647-b54a-bfc93eed5e43
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss,Accuracy
500,1.539300,1.384419,0.553500
1000,1.205800,1.038601,0.671500
1500,0.752800,0.724212,0.818500
2000,0.513200,0.522842,0.864000
2500,0.294000,0.418670,0.885500
3000,0.234400,0.361844,0.889500
3500,0.152300,0.335328,0.892500
4000,0.143900,0.323098,0.890000
4500,0.098800,0.318395,0.895500
5000,0.099700,0.311260,0.892000


[I 2023-04-29 10:20:26,814] Trial 0 finished with value: 0.892 and parameters: {'learning_rate': 0.00046486201634241193, 'batch_size': 16, 'num_train_epochs': 2}. Best is trial 0 with value: 0.892.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss,Accuracy
500,1.631100,1.508010,0.482500
1000,1.368700,1.315691,0.593000
1500,1.094500,1.087030,0.684500
2000,0.819000,0.878279,0.785000
2500,0.598000,0.714038,0.842500
3000,0.439400,0.593007,0.863500
3500,0.330400,0.507852,0.876500
4000,0.255700,0.448025,0.884500
4500,0.203400,0.405967,0.890500
5000,0.165700,0.375982,0.894000


[I 2023-04-29 10:21:43,305] Trial 1 finished with value: 0.8955 and parameters: {'learning_rate': 0.00013029306778240707, 'batch_size': 128, 'num_train_epochs': 4}. Best is trial 1 with value: 0.8955.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


...

[I 2023-04-29 10:50:52,412] Trial 17 finished with value: 0.875 and parameters: {'learning_rate': 3.608198091244232e-05, 'batch_size': 64, 'num_train_epochs': 2}. Best is trial 15 with value: 0.8965.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss,Accuracy
500,1.503300,1.312648,0.550500
1000,1.087800,0.886244,0.767500
1500,0.693300,0.579409,0.844500
2000,0.508800,0.417828,0.880000
2500,0.211700,0.357517,0.884000
3000,0.176600,0.338776,0.886000
3500,0.173700,0.323820,0.888500
4000,0.205000,0.321999,0.889500
4500,0.085600,0.322655,0.891500
5000,0.097600,0.323908,0.890000


[I 2023-04-29 10:51:14,714] Trial 18 finished with value: 0.8895 and parameters: {'learning_rate': 0.0009283588829064944, 'batch_size': 8, 'num_train_epochs': 3}. Best is trial 15 with value: 0.8965.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss,Accuracy
500,1.447300,1.196647,0.632000
1000,0.743500,0.664002,0.856000
1500,0.323500,0.441980,0.885500
2000,0.175700,0.361310,0.893000
2500,0.113900,0.328618,0.894500
3000,0.081400,0.315929,0.894500
3500,0.061600,0.314411,0.894000
4000,0.048400,0.318799,0.894000
4500,0.039000,0.325821,0.895000
5000,0.032100,0.334757,0.895000


[I 2023-04-29 10:51:58,108] Trial 19 finished with value: 0.894 and parameters: {'learning_rate': 0.0003385941209472733, 'batch_size': 128, 'num_train_epochs': 4}. Best is trial 15 with value: 0.8965.


### 3.3. Evaluation on test set

In [9]:
# Your code to evaluate the final model on the test set here
eval_results = trainer.evaluate(dset_tokenized["validation"])

print(eval_results)

{'eval_loss': 0.32368600368499756, 'eval_accuracy': 0.8915, 'eval_runtime': 0.3484, 'eval_samples_per_second': 5740.432, 'eval_steps_per_second': 717.554, 'epoch': 108.0}


Having fun :)

In [10]:
# creates a dataset with one example
def create_example(texts):
  labels = [];
  for item in texts:
    labels.append(0);
  data = {
      'text': texts,
      'label': labels,
  }
  return datasets.Dataset.from_dict(data);

In [11]:
# Testing random sentences that could be tweeted to see what they give as the result :)
texts = ['wow last night was so cool would recommend to everyone',
         'jesus the concert last night was so bad',
         'cant stand to stay at the school with these annoying people',
         'just got an F on the test',
         'just got an A on the test',
         'wow i never tought this day would come',
         'I just feel so useless every day',
         'wow finally feel like people really like me',
         'Just came from a date and it was so fun. He seems so nice and I can\'t wait to call him again',
         'i have been having a really good time with my friends and i feel like they really do care about me'
         'wait what just happened. omd I can\'t believe my eyes',
         'i was really suprised with the results',
         'tomorrow i have to go meet with my ex boyfriend. I just can\'t stand him anymore hes such an asshole',
         'he just suddenly fell infront of me I couldn\'t believe it',
         'I love him so much that I can\'t wait to marry him'];
labels = ['sadness', 'joy', 'love', 'anger', 'fear', 'suprise'];
example = create_example(texts); #creates a dict with just the example
example_tokenized = example.map(vectorize_item, num_proc=4); # tokenize
for i in range(len(texts)):
  print(texts[i])
  prediction = trainer.predict(example_tokenized).predictions[i]; # predicts the label
  largest = max(prediction); # label with largest value
  labelOfLargest = labels[list(prediction).index(largest)]; # name of label
  print(f"{labelOfLargest} with confidence of: {largest}\n");

Map (num_proc=4):   0%|          | 0/14 [00:00<?, ? examples/s]

wow last night was so cool would recommend to everyone


joy with confidence of: 3.9040849208831787

jesus the concert last night was so bad


sadness with confidence of: 2.8583946228027344

cant stand to stay at the school with these annoying people


anger with confidence of: 0.6225325465202332

just got an F on the test


...

joy with confidence of: 2.8397653102874756

i was really suprised with the results


joy with confidence of: 1.1881299018859863

tomorrow i have to go meet with my ex boyfriend. I just can't stand him anymore hes such an asshole


fear with confidence of: 1.553254246711731

he just suddenly fell infront of me I couldn't believe it


sadness with confidence of: 1.22055184841156

I love him so much that I can't wait to marry him


joy with confidence of: 1.9503124952316284



---

## 4. Results and summary

### 4.1 Corpus insights

(Briefly discuss what you learned about the corpus and its annotation)

The corpus includes Twitter messages in english and they have been annotated with six basic emotions which are anger, fear, joy, love, sadness, and surprise.
By the paper the tweets have been selected with some hashtags and then annotated. The selected hastags can be seen from the paper.

If i understood correctly the tweets are gathered by hastags and these hastags define the emotion that is in the tweet for example #worried will be a tweet that displays fear and #fun will be joy and so on. So this has been annotated with distant supervision


### 4.2 Results

(Briefly summarize your results)

The percentage that I got was at best times in the test just over 90% and a little bit less in validation (ab 89-90%). 

Had some fun and tried the model on "tweets" that I came up with and saw some interesting results. As you can see it does hit the mark on some of the tests, but also alot of the results are so and so. For example "he just suddenly fell infront of me I couldn't believe it" and "wow finally feel like people really like me" are labeled sadness and joy. While yes I feel like these can be these labels, but I would label these as suprise and love. So while the results are fine sometimes they feel like they take the "easier" result.

After looking into the dataset I noticed that exactly these (love and suprise) are a bit rarer than the other categories. Especially suprise was also the lowest accuracy label in the paper. Also alot of the "tweets" that were regarding love and suprise were in my opinion a bit suprising and I would have maybe categorized them as something else. So this will effect the results when I use just tweets that I come up with my self and that aren't the same type as in the train data. I feel like this is also the 10% that is lacking in my model and it fails mostly with these labels.

I think joy and sadness are the two feelings that are predicted really well in the model as they seem to work with my own tweets also. This would be also logical as the feeling are a bit vague and fit a lot of different sentences. But they are also the two most represented feelings in the dataset.

Something else that is interesting in my opinion is the tweets "just got an A on the test" and "just got an F on the test" that were labeled as joy and fear. Was really happy to see this and proves that the machine actually learnt something by understanding that getting a F on a test is bad and getting an A on the test is good. :)

### 4.3 Relation to state of the art

(Compare your results to the state-of-the-art performance)

The state of the art models are in the 95% range and my model is just closing in on 90% depending on luck of the model at times.

Not bad in my opinion when taking into consideration how simple this model is and the models that reach 95% seem to be BERT based and a lot larger.

---

## 5. Bonus Task (optional)

### 5.1. Annotating out-of-domain documents

(Briefly describe the chosen out-of-domain documents)

Not the easiest to select a new dataset for this because most of the documents for example news are commonly very neutral and don't showcase any distinct emotions. I selected dialogue and found a huggingface dataset of empathetic dialogue.

(Briefly describe the process of annotation)

My idea for the annotation is to loop 50 timest the dataset and annotate every example that comes up and then add it to a new dataset. This would be pretty easy and I would be able to add it straight into a dataset.

### 5.2 Conversion into dataset

In [12]:
# load the imdb dset
eval_dset = datasets.load_dataset('empathetic_dialogues');
# check it works
print(eval_dset);

Generating train split:   0%|          | 0/76673 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/12030 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10943 [00:00<?, ? examples/s]

Dataset empathetic_dialogues downloaded and prepared to /root/.cache/huggingface/datasets/empathetic_dialogues/default/0.1.0/09bbeed3882a67db98c73952fb3c1c9a85af83dc78f81454c2454382fd03f6cf. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['conv_id', 'utterance_idx', 'context', 'prompt', 'speaker_idx', 'utterance', 'selfeval', 'tags'],
        num_rows: 76673
    })
    validation: Dataset({
        features: ['conv_id', 'utterance_idx', 'context', 'prompt', 'speaker_idx', 'utterance', 'selfeval', 'tags'],
        num_rows: 12030
    })
    test: Dataset({
        features: ['conv_id', 'utterance_idx', 'context', 'prompt', 'speaker_idx', 'utterance', 'selfeval', 'tags'],
        num_rows: 10943
    })
})


In [13]:
# remove all but unsupervised
del eval_dset["validation"];
del eval_dset["test"];


In [14]:
eval_dset = eval_dset.shuffle() #shuffle the dataset for safety

In [15]:
labels = ['sadness', 'joy', 'love', 'anger', 'fear', 'suprise']; # to remember the labels
texts = []; # create a empty list.
labels = []; # empty labels list
count = 0;
for item in eval_dset['train']: # loop first 50
  print(item['prompt']); # show text and get sentiment
  sent = int(input('give sentiment to the text: '))
  texts.append(item['prompt']);
  labels.append(sent);
  count += 1;
  if count >= 50: break;

I'm impressed with my new used car that I bought for cheap. It runs great.
give sentiment to the text: 1
I leave for vacation on Sunday. I have all my bags packed and have everything I need.
give sentiment to the text: 1
My neighbor hit on my wife!
give sentiment to the text: 3
I was out with my guy friends last weekend. A girl approached me and wanted my number but I told her no because I have a girlfriend.
give sentiment to the text: 5
My family threw me a surprise party this year for my birthday. I had no idea they were planning it_comma_ and even some of my relatives living overseas came to visit.
give sentiment to the text: 1
I was mad when I almost got evicted. It was the landlords fault
give sentiment to the text: 3
I miss my gramma so much
give sentiment to the text: 0
I really wanted this thing from a store when I was little. Because of this I stole it and I felt really bad about it
give sentiment to the text: 0
I was really excited when my daughter came home from college to v

In [16]:
data = {
  'text': texts,
  'label': labels,
}
dset_dict = datasets.Dataset.from_dict(data); #create a dataset from the examples
tokenized_eval_dset = dset_dict.map(vectorize_item,num_proc=4); #tokenize the dataset

Map (num_proc=4):   0%|          | 0/50 [00:00<?, ? examples/s]

In [17]:
print(tokenized_eval_dset[0]); # check it worked

{'text': "I'm impressed with my new used car that I bought for cheap. It runs great.", 'label': 1, 'input_ids': [1560, 1930, 2159, 5240, 5793, 6495, 6625, 7044, 8002, 8777, 8946, 11419, 13439, 14331, 14933]}


### 5.3. Model evaluation on out-of-domain test set

In [18]:
# Your code to evaluate the model on the out-of-domain test set here
res = trainer.evaluate(tokenized_eval_dset);

print(res);

{'eval_loss': 1.585536241531372, 'eval_accuracy': 0.48, 'eval_runtime': 0.0375, 'eval_samples_per_second': 1332.735, 'eval_steps_per_second': 186.583, 'epoch': 108.0}


### 5.4 Bonus task results

(Present the results of the evaluation on the out-of-domain test set)
Well... So got a result of 48%. And was maybe waiting for a bit of a better result but this is what we got. The text was kinda similar because tweets are pretty similar to spoken language.

But I feel like my annotation of the data is fairly different from the annotation of the original dataset and annoting it yourself brought some light into the difficulty of annotation. There were a lot of so and so prompts that could be multiple values and also really weren't any of the six emotions. Also my own view of life seemed to attribute into the emotions I selected. For example there were some prompts of getting items stolen or recieving bad food. I would myself feel sadness from these, but different people could also feel anger. Also I would describe nostalgic feelings positive and with joy but some may feel sadness from listening to an old song... So I see where the drop of eval comes from.

### 5.5. Annotated data

In [19]:
# Include your annotated out-of-domain data here
for i in dset_dict:
  print(i);

{'text': "I'm impressed with my new used car that I bought for cheap. It runs great.", 'label': 1}
{'text': 'I leave for vacation on Sunday. I have all my bags packed and have everything I need.', 'label': 1}
{'text': 'My neighbor hit on my wife!', 'label': 3}
{'text': 'I was out with my guy friends last weekend. A girl approached me and wanted my number but I told her no because I have a girlfriend.', 'label': 5}
{'text': 'My family threw me a surprise party this year for my birthday. I had no idea they were planning it_comma_ and even some of my relatives living overseas came to visit.', 'label': 1}
{'text': 'I was mad when I almost got evicted. It was the landlords fault', 'label': 3}
{'text': 'I miss my gramma so much', 'label': 0}
{'text': 'I really wanted this thing from a store when I was little. Because of this I stole it and I felt really bad about it', 'label': 0}
{'text': 'I was really excited when my daughter came home from college to visit me.', 'label': 1}
{'text': 'I sta